In [ ]:
%load_ext aiida
%aiida

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm import StructureData, Dict, FolderData, WorkChainNode
from aiida.orm import Str

from aiida.common.exceptions import NotExistent

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

import matplotlib.pyplot as plt
from pprint import pprint

import nglview

import ase.io
import tempfile

import re
import numpy as np
import scipy.constants

from collections import OrderedDict

# Nudged Elastic Band calculations

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png')
    raw = open(tmp.name, 'rb').read()
    tmp.close()
    return b64encode(raw).decode()

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))


#viewer = nglview.NGLWidget()
#
#style = {'description_width': '120px'}
#layout = {'width': '70%'}
#slider_image_nr = ipw.IntSlider(description='image nr.:',
#                              value=1, step=1,
#                              min=1, max=2,
#                              style=style, layout=layout)
#
#
all_ase=[]
#
#def on_image_nr_change(c):
#    visualized_ase = all_ase[slider_image_nr.value-1]
#    refresh_structure_view(visualized_ase)
#slider_image_nr.observe(on_image_nr_change, 'value')    
#
#clear_output()
#display(ipw.VBox([slider_image_nr, viewer]))

In [ ]:
def make_replica_html(structure_data_list, energies, distances):
    html = '<table>'
    
    n_col = 4
    for i, (rep, en, dist) in enumerate(zip(structure_data_list, energies, distances)):
        thumbnail = rep.get_extra('thumbnail')
        # The table cell
        if i%n_col == 0:
            html += '<tr>'
        html += '<td><img width="400px" src="data:image/png;base64,{}" title="">'.format(thumbnail)
        # Output some information about the replica...
        html += '<p><b>Nr: </b>{} <br> <b>Energy:</b> {:.6f} eV <br> <b>Dist. to prev:</b> {:.4f} ang</p>'\
                .format(i, en, dist)
        html += '<p>pk: {}</p>'.format(rep.pk)
        # ... and the download link.
        html += '<p><a target="_blank" href="export_structure.ipynb?uuid={}">View & export</a></p><td>'\
                .format(rep.uuid)
        if i%n_col == n_col-1:
            html += '</tr>'
            
    html += '</tr>'
    html += '</table>'
    return html

In [ ]:
def sorted_opt_rep_keys(keys):
    return sorted([ (int(key.split('_')[2]), key) for key in keys if 'opt_replica' in key])

def process_and_show_neb(c):
    global all_ase
    
    wc=load_node(node.value)
    structure_data_list = []
    
    btn_show.disabled = True
    with main_out:
        clear_output()
            
    for i_rep in range(wc.inputs.neb_params['number_of_replica']):
        label = "opt_replica_%s" % str(i_rep).zfill(3)
        structure_data_list.append(wc.outputs[label])
        
    energies_array = wc.outputs['replica_energies'].get_array('energies') * 27.211386245
    distances_array = wc.outputs['replica_distances'].get_array('distances') * 0.529177
    
    energies_array = np.array([e_arr - e_arr[0] for e_arr in energies_array])
        
    #### --------------------------------------------------------------
    ## Add thumbnails to replicas if they are not already added
    ## ans store list of ASE structures for the viz
    for rep in structure_data_list:
        the_ase=rep.get_ase()
        all_ase.append(the_ase)
        if not "thumbnail" in rep.extras:
            rep.set_extra("thumbnail", render_thumbnail(the_ase))
    #### --------------------------------------------------------------
    
    replica_html = make_replica_html(structure_data_list, energies_array[-1], distances_array[-1])
    
    barrier_list = [np.max(e_arr) for e_arr in energies_array]
    
    with main_out:
        f, axarr = plt.subplots(1, 2, figsize=(14, 4))
        
        axarr[0].plot(energies_array[-1], 'o-')
        axarr[0].set_ylabel("Energy (eV)")
        axarr[0].set_xlabel("Replica nr")
        axarr[0].set_title("NEB energy profile")
        
        axarr[1].plot(barrier_list, 'o-')
        axarr[1].axhline(barrier_list[-1], linestyle='--', color='lightgray')
        axarr[1].set_ylabel("Barrier (eV)")
        axarr[1].set_xlabel("Iteration nr")
        axarr[1].set_title("NEB convergence")
        
        plt.show()
        
        display(ipw.HTML(replica_html))
        
        print("List of all replica PKs:")
        rep_pk_str = "["
        for struct in structure_data_list:
            rep_pk_str += "%d " % struct.pk
        print(rep_pk_str[:-1] + "]")
    
    
    btn_show.disabled = False
    #slider_image_nr.max=len(all_ase)
    #initialize_structure_view()

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

node=ipw.IntText(description='Load node: ', layout=layout, style=style)

btn_show = ipw.Button(description="Show")
btn_show.on_click(process_and_show_neb)

main_out = ipw.Output()

display(node,btn_show, main_out)